# Doc2Vecの動作確認（bot_id=11）


実行条件は下記の通りとします。

- すべての教師データのタグをユニークにするため、[ラベル]\_[連番] 形式のタグを付与


- 品詞を落とさないようにする


- DBOWを使用


- feature数は、実質的な教師データのパターン数と整合させる（ユーザーさんによっては、回答ラベル数が必ずしも回答パターン数と合致しないため、ラベル数の２倍程度の値を設定するのを目安とします）


- 学習時の反復回数を [feature数 * 10] に設定


現在テストデータとして用意されているBot（ID=11：benefitone.csv）で動作確認を行います。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) 学習／予測処理

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、一部の品詞を落とすようにします。

（＝learning.core.nlang.Nlang クラスの仕様に従います）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 11
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/21 PM 11:23:50 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/21 PM 11:23:50 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [5]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang_naive.batch_split(_sentences)

### (2-2) タグ付け

ラベル毎の教師データ数を制限した学習セットを構築します。

In [6]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

class Doc2VecTrainingSet:
    def __init__(self):
        self.answer_id_count = {}
        self.selected_answer_ids = []
        self.selected_questions = []
        self.selected_separated_sentences = []

    def build(self, questions, answer_ids, separated_sentences):
        for index, answer_id in enumerate(answer_ids):
            if answer_id not in self.answer_id_count.keys():
                self.answer_id_count[answer_id] = 0

            '''
                データをリストに格納
                ラベルはユニークにする必要があるので、
                [answer_id]_[連番] の形式で編集
            '''
            self.selected_answer_ids.append('%04d_%02d' % (
                answer_id,
                self.answer_id_count[answer_id]
            ))
            self.selected_questions.append(questions[index])
            self.selected_separated_sentences.append(separated_sentences[index])
            self.answer_id_count[answer_id] += 1
        
        return self
    
    def get_tagged_document_list(self):
        '''
            ユニークになったコーパス／ラベルから、
            学習セットを生成する
        '''
        tagged_document_list = self.__corpus_to_sentences(
            self.selected_separated_sentences, 
            self.selected_answer_ids
        )

        return tagged_document_list

    def __get_tagged_document(self, sentences, name):
        '''
            models.doc2vecの仕様に従い
            コーパスにタグ付け
        '''
        words = sentences.split(' ')
        return TaggedDocument(words=words, tags=[name])

    def __corpus_to_sentences(self, separated_sentences, answer_ids):
        '''
            TaggedDocumentを生成し、リストに格納
        '''
        tagged_document_list = []
        for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
            tagged_document = self.__get_tagged_document(doc, name)
            tagged_document_list.append(tagged_document)
            
        return tagged_document_list

In [7]:
'''
    ユニークになったコーパス／ラベルから、
    学習セットを生成する
    
    ラベル数＝88、サンプル数＝7,083
'''
d2v_training_set = Doc2VecTrainingSet()
d2v_training_set.build(questions, answer_ids, _separated_sentences)

### (2-3) 学習処理／モデルのシリアライズ

In [8]:
def doc2vec_model_path(bot_id):
    model_path = 'prototype/better_algorithm/doc2vec.bot%02d.model' % bot_id

    return model_path

def train_by_doc2vec(bot_id, doc2vec_training_set):
    '''
        パラメータ：
         学習時にDBOWを使用する
         featureの数を回答パターン数の２倍（仮決め）に設定
         反復回数をfeature数の10倍に設定
    '''
    sentence_list = doc2vec_training_set.get_tagged_document_list()
    n_pattern = len(doc2vec_training_set.answer_id_count)
    print('train_by_doc2vec: Train data sample=%d, Train data pattern=%d' % (len(sentence_list), n_pattern))

    n_feature = n_pattern * 2
    n_iter = n_feature * 10
    print('train_by_doc2vec: Feature size=%d, Max iteration count=%d' % (n_feature, n_iter))

    # ボキャブラリ生成／学習実行
    model = Doc2Vec(dm=0, size=n_feature, min_count=1, iter=n_iter)
    model.build_vocab(sentence_list)
    ret = model.train(sentence_list)

    '''
        モデル内に保持されているベクトルの数を取得
        （featureの数 [回答パターン数の２倍] と同じであることを確認）
    '''
    if len(model.docvecs) != len(sentence_list):
        raise Exception('train_by_doc2vec: Failed to create document vector')

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(bot_id))
    print('train_by_doc2vec: document vector size=%d, return=%d' % (len(model.docvecs), ret))

    return ret

In [9]:
'''
    生成された学習セット（タグ付きドキュメント）を
    使用し、学習実行
'''
train_by_doc2vec(_bot_id, d2v_training_set)

train_by_doc2vec: Train data sample=7083, Train data pattern=88
train_by_doc2vec: Feature size=176, Max iteration count=1760
train_by_doc2vec: document vector size=7083, return=60768125


60768125

## (3) accuracy 測定

In [10]:
def predict_similarity(separated_sentence):
    corpus = separated_sentence.split()
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    answer_id, similarity = ret[0]
    return corpus, answer_id, similarity

def get_prediction_statistics(separated_sentences, answer_ids):
    '''
        学習セットの質問文をそのまま予測処理にかけて、
        回答を予測
    '''
    statistics = []
    for i, _ in enumerate(separated_sentences):
        sentence = separated_sentences[i]
        preferred_answer_id = answer_ids[i]
        corpus, answer_id, similarity = predict_similarity(separated_sentences[i])
        corpus_len = len(corpus)
        statistics.append((i, corpus_len, preferred_answer_id, answer_id, similarity))

    return statistics

In [11]:
def calculate_accuracy(prediction_statistics):
    '''
        予測結果を、質問文の単語数毎／回答ID毎に統計する
    '''
    ncorrect_by_corpus_len = {}
    nsample_by_corpus_len = {}

    ncorrect = 0
    nsample = 0

    for statistics in prediction_statistics:
        i, corpus_len, preferred_answer_id, answer_id, similarity = statistics

        '''
            質問文の単語数ごとに統計を取る
        '''
        if corpus_len not in nsample_by_corpus_len.keys():
            ncorrect_by_corpus_len[corpus_len] = 0
            nsample_by_corpus_len[corpus_len] = 0
        nsample_by_corpus_len[corpus_len] += 1

        '''
            正解かどうか検査
            （NNNN_nn 形式ラベルの上４桁が一致していれば正解とします）
        '''
        nsample += 1
        if preferred_answer_id[0:4] == answer_id[0:4]:
            ncorrect += 1
            ncorrect_by_corpus_len[corpus_len] += 1
    
    '''
        全体の正解率
    '''
    accuracy = ncorrect / nsample
    print("accuracy=%0.3f (%d/%d)" % (accuracy, ncorrect, nsample))

    '''
        質問文の単語数ごとの統計情報を編集
    '''
    info_by_corpus_len = []
    for k, v in ncorrect_by_corpus_len.items():
        info_by_corpus_len.append((
            k, 
            ncorrect_by_corpus_len[k]/nsample_by_corpus_len[k], 
            ncorrect_by_corpus_len[k], 
            nsample_by_corpus_len[k]
        ))

    '''
        質問文の単語数ごとの正解率をリスト
    '''
    print("Accuracy info by word count...")
    for info in info_by_corpus_len:
        print("word_count=%2d: accuracy=%0.3f (%d/%d)" % (
            info[0], info[1], info[2], info[3]
        ))

    return accuracy

In [12]:
loaded_model = models.Doc2Vec.load(doc2vec_model_path(_bot_id))

selected_separated_sentences = d2v_training_set.selected_separated_sentences
selected_answer_ids = d2v_training_set.selected_answer_ids

In [13]:
prediction_statistics = get_prediction_statistics(selected_separated_sentences, selected_answer_ids)
calculate_accuracy(prediction_statistics)

accuracy=0.968 (6854/7083)
Accuracy info by word count...
word_count= 1: accuracy=0.125 (1/8)
word_count= 2: accuracy=1.000 (40/40)
word_count= 3: accuracy=0.990 (301/304)
word_count= 4: accuracy=0.944 (739/783)
word_count= 5: accuracy=0.971 (1355/1396)
word_count= 6: accuracy=0.964 (1174/1218)
word_count= 7: accuracy=0.969 (1122/1158)
word_count= 8: accuracy=0.962 (802/834)
word_count= 9: accuracy=0.992 (510/514)
word_count=10: accuracy=0.981 (257/262)
word_count=11: accuracy=0.981 (204/208)
word_count=12: accuracy=0.978 (135/138)
word_count=13: accuracy=0.979 (92/94)
word_count=14: accuracy=0.952 (59/62)
word_count=15: accuracy=1.000 (28/28)
word_count=16: accuracy=1.000 (14/14)
word_count=17: accuracy=1.000 (12/12)
word_count=18: accuracy=1.000 (6/6)
word_count=19: accuracy=1.000 (2/2)
word_count=23: accuracy=0.500 (1/2)


0.9676690667796132

## (4) 予測処理

In [14]:
def predict(word, bot_id):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(doc2vec_model_path(bot_id))
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

### (4-1) nosetestsにあった質問文

正しく回答することができているようです。

In [15]:
'''
    契約書を見たいのですが（正解＝4683）
'''
predict('契約書を見たいのですが', _bot_id)

(['契約', '書', 'を', '見る', 'たい', 'の', 'です', 'が'],
 [('4683_25', 0.8374380469322205),
  ('4683_12', 0.8345401287078857),
  ('4683_00', 0.8338999152183533),
  ('4683_24', 0.8318637609481812),
  ('4683_15', 0.7280780076980591),
  ('4683_03', 0.7241529226303101),
  ('4683_11', 0.6865381002426147),
  ('4683_23', 0.6797800064086914),
  ('4683_16', 0.6764998435974121),
  ('4683_08', 0.6756047010421753)])

In [16]:
'''
    EXカードを貸してください（正解＝4678）
'''
predict('EXカードを貸してください', _bot_id)

(['ＥＸ', 'カード', 'を', '貸す', 'て', 'くださる'],
 [('4678_692', 0.8838238716125488),
  ('4678_2700', 0.8837131261825562),
  ('4678_686', 0.7681745886802673),
  ('4741_16', 0.765505313873291),
  ('4678_2416', 0.7654439210891724),
  ('4741_07', 0.7652233839035034),
  ('4678_2694', 0.763477087020874),
  ('4740_12', 0.7615160346031189),
  ('4678_408', 0.7611214518547058),
  ('4740_24', 0.7568632364273071)])

### (4-2) 教師データに存在する、単語１語だけの質問文

三脚が含まれる質問文が候補としてリストされます。類似度が0.5を上回る結果となります。

（ちなみに確認したところ、4740と4741は同じ「三脚を借りたいのですね。総務部に・・・」という回答文でした）

In [17]:
'''
    三脚（正解＝4740）
'''
predict('三脚', _bot_id)

(['三脚'],
 [('4740_25', 0.7282403707504272),
  ('4740_00', 0.725811243057251),
  ('4741_19', 0.7173864841461182),
  ('4741_00', 0.7127325534820557),
  ('4740_23', 0.7092810869216919),
  ('4741_07', 0.7047088146209717),
  ('4741_16', 0.7031413316726685),
  ('4740_11', 0.7005681991577148),
  ('4740_24', 0.6974652409553528),
  ('4740_12', 0.6924776434898376)])

### (4-3) 教師データに存在しない、単語１語だけの質問文

類似度が0.5を下回る結果となります。

In [18]:
'''
    こんにちは（正解＝なし）
'''
predict('こんにちは', _bot_id)

(['こんにちは'],
 [('4678_777', 0.23522378504276276),
  ('4678_2785', 0.23461759090423584),
  ('4678_3808', 0.2279362976551056),
  ('4678_1800', 0.21939142048358917),
  ('4776_01', 0.21780624985694885),
  ('4678_3191', 0.2155098021030426),
  ('4776_10', 0.21515773236751556),
  ('4678_1183', 0.21403685212135315),
  ('4782_01', 0.21228154003620148),
  ('4678_2967', 0.21169403195381165)])

### (4-4) 正解は教師データにないが、一部品詞が合致している質問文

4837、4683、4678、4713といったところが候補となり、類似度が0.5を上回る結果となります。

In [19]:
'''
    おいしいラーメンが食べたいです（正解＝なし）
'''
predict('おいしいラーメンが食べたいです', _bot_id)

(['おいしい', 'ラーメン', 'が', '食べる', 'たい', 'です'],
 [('4837_19', 0.7025206685066223),
  ('4837_00', 0.7010031342506409),
  ('4683_00', 0.6923429369926453),
  ('4683_25', 0.6840815544128418),
  ('4678_3850', 0.6818233728408813),
  ('4713_15', 0.6809757947921753),
  ('4713_06', 0.6804230809211731),
  ('4678_194', 0.6702786087989807),
  ('4678_4423', 0.6693686246871948),
  ('4678_3181', 0.6687726974487305)])

#### ご参考：教師データと一部品詞が合致する例

- '使いたいソフトがある'（正解＝4837）['使う', 'たい', 'ソフト', 'が', 'ある']

- '契約書が見たい'（正解＝4683）['契約', '書', 'が', '見る', 'たい']

- '製本テープが使いたい'（正解＝4678）['製本', 'テープ', 'が', '使う', 'たい']

- 'イントラが見たい'（正解＝4713）['イントラ', 'が', '見る', 'たい']

### (4-5) 正解が教師データに存在せず、合致する品詞も無い質問文

類似度は、0.5を大幅に下回る結果となります。

In [20]:
'''
    正解無し質問探索（正解＝なし）
'''
predict('正解無し質問探索', _bot_id)

(['正解', '無し', '質問', '探索'],
 [('4678_1276', 0.26643067598342896),
  ('4678_3284', 0.2660854756832123),
  ('4687_03', 0.22695830464363098),
  ('4687_12', 0.22388872504234314),
  ('4678_915', 0.2128073275089264),
  ('4680_17', 0.2060241997241974),
  ('4686_21', 0.20368823409080505),
  ('4686_09', 0.20230400562286377),
  ('4678_2885', 0.2001335769891739),
  ('4678_2923', 0.1999267339706421)])

#### ご参考：１語でも合致する品詞が含まれた場合の類似度

「の」という品詞を含めただけで、類似度が0.5以上に跳ね上がる結果となってしまいます。

In [21]:
'''
    正解無し質問の探索（正解＝なし）
'''
predict('正解無し質問の探索', _bot_id)

(['正解', '無し', '質問', 'の', '探索'],
 [('4678_05', 0.6415265202522278),
  ('4733_19', 0.6386672258377075),
  ('4733_00', 0.6373109817504883),
  ('4678_4234', 0.6355193853378296),
  ('4678_4261', 0.6328074336051941),
  ('4678_3223', 0.6318308115005493),
  ('4678_4239', 0.6301361322402954),
  ('4678_174', 0.6300672292709351),
  ('4678_267', 0.629685640335083),
  ('4678_4403', 0.626211941242218)])